## PyPSA Introduction Notebook

<div style="display: flex; align-items: center; justify-content: space-between; gap: 1rem; flex-wrap: wrap;">

  <div style="flex: 1; min-width: 250px;">
    <p><strong>Author:</strong><br>
    Priyesh Gosai – Energy Systems Modeler and Training Coordinator</p>
    <p><strong>Contributors:</strong><br>
    Daniel Rüdt – Energy Systems Modeler<br>
    Thomas Gilon – Energy Systems Modeler</p>
  </div>

  <div style="flex: 0 0 auto;">
    <a href="https://openenergytransition.org/index.html" target="_blank">
      <img src="https://openenergytransition.org/assets/img/oet-logo-red-n-subtitle.png" height="60" alt="Open Energy Transition logo">
    </a>
  </div>

</div>


### ⚙️ Setup Environment

**💻 Running in Google Colab**

This notebook is designed to run in **Google Colab**, which provides access to a cloud-based virtual machine. While this is convenient and requires no local setup, it's important to note that **Colab sessions are temporary** — all files and installations are **lost when the session ends**.

To ensure your work is preserved:

- The notebook is set up to **link with your Google Drive**. This allows you to save input files, results, and any model modifications to your own Drive for later use.

- The notebook is also linked to a **GitHub repository**. You can choose whether or not to **pull the latest version of the repository** each time you run the notebook using the boolean options provided below.

  - If enabled, the notebook will:
    - Pull the latest version of the repository into your Drive.
    - Create a backup folder containing your existing work, timestamped for traceability.
    - To avoid cluttering your Google Drive, remember to delete old backups or set the `backup_old_repo = False` option.

Another consideration when using Colab is that **PyPSA and several required packages are not pre-installed** in the Colab environment. These packages must be installed manually each time you connect to a new VM.

For both the GitHub sync and the package installation process, we’ve provided boolean flags that let you **skip these steps** if desired. This gives you more control over the setup process, especially when restarting work or debugging.



**Setup Environment**
1. Provide the path to the repository for this project.
2. Confirm if this is the first run in the Google Colab environment.
3. Confirm if you want to update the repository with the latest changes.

In [ ]:
repo_path  = f'https://github.com/open-energy-transition/EIS-2025'
first_run = True # Set to True if this is the first run of the in the Google Colab environment
update_repo = True # Set to True if you want to update the repository with the latest changes

In [ ]:
# @title Connect this Notebook to your Google Drive
# @markdown Run this cell to link this notebook to the GitHub repository. <br>
# @markdown If the repository already exists, it will back up any modified files and force pull the latest changes.
from google.colab import drive
import os
import subprocess
import shutil
from datetime import datetime
import urllib.request

# Mount Google Drive
drive.mount('/content/drive')

# Base directory
base_dir = '/content/drive/MyDrive/'
os.chdir(base_dir)

FOLDER = os.path.basename(repo_path)


if update_repo:


    # Get timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    backup_folder = f'Backup_{timestamp}'
    backup_path = os.path.join(base_dir, backup_folder)

    # If repo exists, back up and force pull
    if os.path.exists(repo_path):
        os.chdir(repo_path)

        # Get list of modified and untracked files
        result = subprocess.run(['git', 'status', '--porcelain'], stdout=subprocess.PIPE, text=True)
        changed_files = [line[3:] for line in result.stdout.splitlines() if line and line[0] in ('M', 'A', '??')]

        if changed_files:
            os.makedirs(backup_path, exist_ok=True)
            print(f"Backing up {len(changed_files)} changed files to {backup_folder}...")
            for file in changed_files:
                src = os.path.join(repo_path, file)
                dst = os.path.join(backup_path, file)
                os.makedirs(os.path.dirname(dst), exist_ok=True)
                if os.path.exists(src):
                    shutil.copy2(src, dst)

        # Force pull: reset local repo to match remote
        subprocess.run(['git', 'fetch', 'origin'])
        subprocess.run(['git', 'reset', '--hard', 'origin/main'])

    else:
        os.chdir(base_dir)
        subprocess.run(['git', 'clone', repo_path])

    
    files = {
        "network_blank.xlsx": "17haq369Y7-5JFRjESfDbUUljVyIEUS4u",  # file ID
        "network_south_africa.xlsx": "1hBqZjotxDuZOQb-kkC0JePbc3gxCGcwq",  # file ID
    }

    for name, file_id in files.items():
        print(f"Retrieving {name} from Google Drive")
        url = f"https://docs.google.com/uc?export=download&id={file_id}"
        urllib.request.urlretrieve(url, name)

    print("Current working directory:", os.getcwd())

In [ ]:
# @title Install the required packages
# @markdown Run this cell to install the necessary Python packages for the project.
if first_run:
    import subprocess, sys, importlib

    packages = [
        "pypsa",        # power‑system modelling & optimization toolbox :contentReference[oaicite:1]{index=1}
        "pypsa[excel]", # pypsa with Excel I/O support
        "folium",       # interactive leaflet‑based maps in Python
        "atlite",       # convert weather reanalysis into energy time‑series :contentReference[oaicite:2]{index=2}
        "pandas",       # tabular data manipulation
        "geopandas",    # pandas + GIS/spatial data support
        "xarray",       # multi‑dim labeled array handling (e.g. gridded data)
        "matplotlib",   # foundational 2D plotting library
        "hvplot",       # high‑level interactive plotting API built on HoloViews
        "geoviews",     # geographic plotting extension for HoloViews
        "plotly",       # interactive plots and dashboards
        "highspy",      # Python wrapper for the high‑performance HiGHS optimizer :contentReference[oaicite:3]{index=3}
        "holoviews",    # declarative visualization library for complex plots
        "mapclassify"   # spatial data classification schemes for choropleth maps
    ]

    subprocess.run(
        [sys.executable, "-m", "pip", "install", "-q", *packages],
        check=True
    )

    missing = [p for p in packages if importlib.util.find_spec(p.split("[")[0]) is None]
    print("✅ All installed!" if not missing else f"❌ Missing: {', '.join(missing)}")


### 🧭 Navigating the PyPSA Object

**Initialize the PyPSA Network** <br>
This cell initializes a PyPSA network  using the meshed AC/DC example.

In [ ]:
import pypsa
import pandas as pd
pd.options.plotting.backend = "plotly"
network = pypsa.examples.ac_dc_meshed()

To create a blank PyPSA `Network`, use:

```python
network = pypsa.Network()
```

To import an existing network from a supported file format (e.g. HDF5, NetCDF, CSV folder, or Excel), pass the file path:

```python
network = pypsa.Network("filename.xx")
```


Display Network Components

In [ ]:
network.components

In [ ]:
# @title Component naming conventions
# @markdown PyPSA follows Python conventions: 
# @markdown   - PascalCase for component classes. 
# @markdown   - snake_case for attributes and methods.


for key in network.component_attrs:
    print(f'{key.ljust(20)} {network.components[key]["list_name"]}')

**Accessing Component Attributes**

We can also access the component attributes directly:

In the next cell, change the `ComponentName` using PascalCase as a string to view the attributes

In [ ]:
ComponentName = 'Generator' 
# ComponentName = 'Bus'
# ComponentName = 'Link'
# ComponentName = 'Line'
# ComponentName = 'Carrier'

network.component_attrs[ComponentName ]

In [ ]:
# @title List of Methods in the Network Object
# @markdown This cell lists all the methods available in the `pypsa.Network` object.
# @markdown You can use these methods to interact with the network object.  
# @markdown The methods are displayed in a grid format for better readability.

import inspect

methods = sorted(
    name for name, func in inspect.getmembers(network, inspect.ismethod)
    if not name.startswith("_")
)

n_cols = 3
n_rows = (len(methods) + n_cols - 1) // n_cols
col_width = max(len(m) for m in methods) + 2

for row_idx in range(n_rows):
    for col_idx in range(n_cols):
        idx = col_idx * n_rows + row_idx
        if idx < len(methods):
            print(methods[idx].ljust(col_width), end="")
    print()


**View the signature, docstring, and source location.**

You can view the description of a specific method by changing the `method_name` variable.

In [ ]:
network.add?

**Explore the Network**

This cell provides an interactive map of the network using the `explore` method using Folium.


In [ ]:
network.plot.explore()

We can create the model without solving it. 

In [ ]:
network.optimize.create_model()

We can view the constraints.

In [ ]:
network.model

**Optimize the Network**

This cell runs the optimization on the network. The default solver is HiGHS, which is a high-performance open-source solver for linear programming problems.

You can change the solver by modifying the `solver_name` parameter in the `network.optimize()` method.

Additional advanced features in PyPSA may be of interest to some users, including support for [custom constraints](https://pypsa.readthedocs.io/en/latest/user-guide/optimal-power-flow.html#custom-constraints), [rolling horizon optimization](https://pypsa.readthedocs.io/en/latest/user-guide/optimal-power-flow.html#rolling-horizon-optimization), and several other modeling options. 

For a full overview, visit the [System Optimization](https://pypsa.readthedocs.io/en/latest/user-guide/optimal-power-flow.html#) section of the PyPSA documentation.


In [ ]:
network.optimize()

In the next section we look at the inputs and outputs for this network.

---

### 🗃️ Data in the PyPSA Object

In [ ]:
# @title Download prepared networks
# @markdown We have prepared a case study that can be explored in this notebook. The case is based on the electircity network from PyPSA-EUR. To allow users to interact with the data we have prepared two networks. One before the optimization and another after the optimization. Running this optimization is a computationally expensive task. 
# @markdown * `pre-network.nc`: The network is prepared before solving it. 
# @markdown * `post-network.nc`: The solved network

from urllib.request import urlretrieve

urls = {
    "pre-network.nc": "https://drive.usercontent.google.com/download?id=17b7YZGXKczY2K5sRPUDJkD5AVwgbOgAh&export=download",
    "post-network.nc": "https://drive.usercontent.google.com/download?id=1qIN0tlZBACPtKCBxHUpBAecBqYsy-sTV&export=download&confirm=t&uuid=cf9cb5cf-de33-4ef4-9f49-5f01f2d571b1",
    }
for name, url in urls.items():
    print(f"Retrieving {name} from Google Drive")
    urlretrieve(url, name)
print("Done")

Import the `pre-network` to an object called `network_eur`

In [ ]:
pre_network_eur = pypsa.Network("pre-network.nc")

Inspect the data in the network. 

_All data in a PyPSA model is stored as pandas DataFrames, allowing you to apply any standard pandas operations directly to PyPSA network components._

In [ ]:
pre_network_eur.buses

In [ ]:
pre_network_eur.generators

In [ ]:
pre_network_eur.generators_t.p_max_pu.head()

In [ ]:
pre_network_eur.generators_t.p_max_pu.plot()

In [ ]:
pre_network_eur.links

In [ ]:
pre_network_eur.lines

In [ ]:
pre_network_eur.loads

If you wanted to solve the network we could run: 
```python
pre_network_eur.optimize(solver_name = 'gurobi') # Assuming that you have a licence. 
```

The solved network data is imported from the file `post-network.nc`

In [ ]:
post_network_eur = pypsa.Network("post-network.nc")

#### 🧮 Results

---

📊 `pypsa.statistics`

The `pypsa.statistics` module provides a set of high-level functions for analyzing and summarizing the results of PyPSA models. It enables users to compute key metrics such as:

- **Generation mix** by technology  
- **Installed capacity** summaries  
- **Line loading** and **congestion metrics**  
- **Costs and revenues** by component  
- **CO₂ emissions** and carbon intensity  
- **Regional energy balances** and **power flows**

These functions are especially useful for post-processing model results and producing plots or reports for scenario analysis, capacity expansion studies, and policy evaluation.

The module works directly with the PyPSA `Network` object and leverages pandas operations internally, so results are returned as DataFrames that can be easily visualized or exported.




In [ ]:
s = post_network_eur.statistics

You can easily have an comprehensive overview of the system level results.

In [ ]:
s().head()

Let's have a look to optimal renewable capacities.

In [ ]:
(
    s.optimal_capacity(
        bus_carrier=["AC", "low voltage"],
        comps="Generator",
    ).div(
        1e3
    )  # GW
)

You can get it as fancy as you want!

In [ ]:
(
    s.optimal_capacity(
        bus_carrier=["AC", "low voltage"],
        groupby=["location", "carrier"],
        comps="Generator",
    )
    .div(1e3)  # GW
    .to_frame(name="p_nom_opt")
    .pivot_table(index="location", columns="carrier", values="p_nom_opt")
    .fillna(0)
    .assign(Total=lambda df: df.sum(axis=1))
    .sort_values(by="Total", ascending=False)
    .round(2)
).head()

We can also easily look into the energy balance for a specific carrier by Node. 

So, let's investigate the Hydrogen balance at the Z1 and Z2 nodes of Germany (DE):

In [ ]:
df = (
    s.energy_balance(groupby=["bus_carrier", "country", "bus", "carrier", "name"])
    .div(1e6)  # TWh
    .to_frame(name="Balance [TWh]")
    .query(
        "(bus_carrier.str.contains('Hydrogen')) "
        "and (country == 'DE') "
        " and (abs(`Balance [TWh]`) > 1e-2)"
    )
    .round(2)
)
df

In [ ]:
# verify energy balance
df.groupby(by="bus").sum()

In [ ]:
exports = df.query("name.str.contains('DE ->')")
export_twh = exports["Balance [TWh]"].sum().round(2)
print(f"DE exports {export_twh} TWh of H2.")

imports = df.query(
    "(name.str.contains('-> DE')) and not (name.str.contains('Z1')) and not (name.str.contains('Z2'))"
)
import_twh = imports["Balance [TWh]"].sum().round(2)
print(f"DE imports {import_twh} TWh of H2.")

balance_twh = import_twh + export_twh
print(
    f"DE is a net {'importer' if balance_twh > 0 else 'exporter'} ({balance_twh.round(2)} TWh)."
)

... or look at renewable curtailment in the system:

In [ ]:
(
    s.curtailment(
        bus_carrier=["AC", "low voltage"],
        groupby=["location", "carrier"],
    )
    .div(1e6)  # TWh
    .to_frame(name="p_nom_opt")
    .pivot_table(index="location", columns="carrier", values="p_nom_opt")
    .fillna(0)
    .assign(Total=lambda df: df.sum(axis=1))
    .sort_values(by="Total", ascending=False)
    .round(2)
).head()

---
📈 `pypsa.plot`

PyPSA also includes a built-in `pypsa.plot` module that provides a small set of standard plotting functions. These are useful for quickly visualizing key aspects of your network, such as:

- Network topology (buses, lines, generators)
- Line loading and power flows
- Generation dispatch over time
- Installed capacities and time-series data

These plots are useful for exploratory analysis and debugging and can be customized using `matplotlib`.

For more advanced or interactive visualizations, you may consider combining PyPSA outputs with external libraries such as `plotly`, `holoviews`, or `hvplot`.


In [ ]:
# let's fill missing colors first
pd.options.plotting.backend = "matplotlib"
post_network_eur.carriers.loc["none", "color"] ="#000000"
post_network_eur.carriers.loc["", "color"] = "#000000"

Let's now plot the optimal renewable capacities that we investigated before.

In [ ]:
s.optimal_capacity.plot.bar(
    bus_carrier="AC",
    query="value>1e3",
    height=6,
);

You can also have details for specific countries.

In [ ]:
s.optimal_capacity.plot.bar(
    bus_carrier="AC",
    query="value>1e3 and country in ['DE', 'FR']",
    height=6,
    facet_col="country",
);

You can have a closer look to the wind production

In [ ]:
s.energy_balance.plot.line(
    facet_row="bus_carrier",
    y="value",
    x="snapshot",
    carrier="wind",
    nice_names=False,
    color="carrier",
    aspect=5.0,
);

... or to the dispatch for specific countries.

In [ ]:
s.energy_balance.plot.area(
    bus_carrier=["AC"],
    y="value",
    x="snapshot",
    color="carrier",
    stacked=True,
    facet_row="country",
    query="country in ['DE', 'FR'] and snapshot < '2013-03'",
    aspect=5,
);

You can also explore H2 results.

In [ ]:
s.energy_balance.plot.bar(
    bus_carrier=["H2"],
    y="carrier",
    x="value",
    color="carrier",
    facet_col="country",
    height=4,
    aspect=1,
    query="country in ['DE', 'FR']",
);

You can also explore the correlation between renewable production and hydrogen.

In [ ]:
s.energy_balance.plot.area(
    bus_carrier=["AC", "H2"],
    y="value",
    x="snapshot",
    color="carrier",
    stacked=True,
    facet_row="bus_carrier",
    sharex=False,
    sharey=False,
    query="snapshot < '2013-03'",
    aspect=5,
);

### 🚀 Getting started with your own PyPSA Model  

* PyPSA v0.34 introduced support for importing models directly from Excel files, where each sheet is named according to PyPSA's component naming conventions.
* This feature provides an easy starting point for building your first PyPSA model.
* This repository includes:
   * A blank Excel template you can use to start your own model.
   * A completed example model of a simple power system for reference.
        * The model uses nine nodes to represent the South African power system.
        * Interconnected using links to simplify the network structure.
As models grow more complex, it's recommended to adopt a more robust and reproducible workflow for long-term development and collaboration.

To import the excel file


```python 
network_name = pypsa.Network('network_name.xlsx')

```

Explore the South African Example

In [ ]:
import pypsa

In [ ]:
network_za = pypsa.Network('network_south_africa.xlsx')

In [ ]:
network_za.loads

In [ ]:
network_za.buses # and similarly for other components

In [ ]:
network_za.optimize() # This will run the optimization on the network

In [ ]:

network_za.export_to_excel('network_export.xlsx') # Export the network to an Excel file


---